# ASF HyP3

> Module for transferring HyP3 processed data to Earth Engine. 

In [ ]:
#| default_exp hyp3

In [ ]:
#| export
import asf_search
import datetime
from IPython.display import JSON
import ee
from fastcore.basics import patch
import gcsfs
from hyp3_sdk import HyP3
import logging
import os
from pprint import pprint
import re
from rio_cogeo import cogeo
import subprocess
import zipfile

from sar_asf_to_gee.core import (
    FORMAT_GEE_DATETIME_STRING,
    create_gee_image_collection
)

# Prerequisites

Authenication for NASA Earthdata
- See: https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/04_NASA_Earthdata_Authentication.html

Authenticate to Google Cloud
- See: https://cloud.google.com/sdk/gcloud/reference/auth/application-default/loginca
- `gcloud auth application-default login`

Authenticate to Google Earth Engine
- See: https://developers.google.com/earth-engine/guides/auth
- `earthengine authenticate`

In [ ]:
hyp3 = HyP3()

In [ ]:
# Set the logging level to display detailed information.
logging.basicConfig(level=logging.INFO)

# Find Files

[HyP3](https://hyp3-docs.asf.alaska.edu/) processing jobs can be initiated in a variety of ways, including the [Vertex](https://hyp3-docs.asf.alaska.edu/using/vertex/) web application and the [HyP3 Python SDK](https://hyp3-docs.asf.alaska.edu/using/sdk/).

The status of previously submitted jobs can be checked on the following page:
https://search.asf.alaska.edu/#/?searchType=On%20Demand

In [ ]:
job_name = 'test submit_insar_job'

batch_completed = hyp3.find_jobs(
    name=job_name,
    status_code='SUCCEEDED',
)

# Limit to one job for simplicity
job = next(batch_completed.__iter__())
display(JSON(job.to_dict()))

# for job in batch_completed:
#     display(JSON(job.to_dict()))

<IPython.core.display.JSON object>

# Transfer completed jobs

In [ ]:
#| export
class Transfer():
    def __init__(
        self,
        job_dict,  # HyP3 job dictionary 
        gcs_bucket,  # GCS bucket
        gee_gcp_project, # GCP project used by Earth Engine
        gee_image_collection=None,  # Name of the Earth Engine ImageCollection (optional)
        local_storage=None,
    ):
        self.job_dict = job_dict
        self.gcs_bucket = gcs_bucket
        self.gee_gcp_project = gee_gcp_project
        self.gee_image_collection = gee_image_collection
        if local_storage:
            self.tempdir = None
            self.local_storage = local_storage
        else:
            self.tempdir = tempfile.TemporaryDirectory() 
            self.local_storage = self.tempdir.name
            logging.debug(f'created temporary directory: {self.tempdir.name}')

In [ ]:
t = Transfer(
    job_dict=job.to_dict(),
    gcs_bucket='hyp3-data-staging',
    gee_gcp_project='sar-asf-to-gee',
    gee_image_collection='test_ic',
    local_storage='temp_downloads',
)

In [ ]:
#| export
@patch
def hpy3_results_to_local(
    self:Transfer,           
):
    "Transfer HyP3 results to local system, unzip, and update the job dictionary."    
    for file in self.job_dict['files']:
        asf_search.download_url(
            url=file['url'],
            path=self.local_storage,
            filename=file['filename'],
        )
        # Unzip the file
        with zipfile.ZipFile(os.path.join(self.local_storage, file['filename']), 'r') as zip_ref:
            zip_ref.extractall(self.local_storage)

        # List the TIF files.
        scene_name = file['filename'].removesuffix('.zip')
        tifs = [x for x in os.listdir(
                    os.path.join('temp_downloads', scene_name))
                if x.endswith('.tif')]

        for tif in tifs:
            logging.info(f'Converting to a Cloud Optimized GeoTIFF: {tif}')
            subprocess.run([
                "rio",
                "cogeo",
                "create",
                os.path.join(self.local_storage, scene_name, tif),
                os.path.join(self.local_storage, scene_name, tif)
            ])
        
        tif_dict = {}
        pattern = rf'^({scene_name}_(.+).tif)$'
        for i in tifs:
            groups = re.search(pattern, i).groups()
            tif_dict[groups[1]] = os.path.join(scene_name, groups[0])
        
        file['extracted'] = tif_dict

In [ ]:
t.hpy3_results_to_local()

# Display the job dictionary, which now includes the list of extracted files.
display(JSON(t.job_dict))

/Users/tylere/Documents/GitHub/gee-community/sar-asf-to-gee/.pixi/env/lib/python3.11/site-packages/asf_search/download/download.py:65: UserWarning: File already exists, skipping download: temp_downloads/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
INFO:root:Converting to a Cloud Optimized GeoTIFF: S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_corr.tif
Reading input: /Users/tylere/Documents/GitHub/gee-community/sar-asf-to-gee/temp_downloads/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_corr.tif

Adding overviews...
Updating dataset tags...
Writing output to: /Users/tylere/Documents/GitHub/gee-community/sar-asf-to-gee/temp_downloads/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_corr.tif
INFO:root:Converting to a Cl

<IPython.core.display.JSON object>

## Transfer to Google Cloud Storage

In [ ]:
#| export
@patch
def to_gcs(
    self:Transfer,
):
    logging.debug('Starting _to_gcs')

    fs = gcsfs.GCSFileSystem(token='google_default')

    for file in self.job_dict['files']:
        for band, filename in file['extracted'].items():
            gcs_path = f'{self.gcs_bucket}/{filename}'
            if fs.exists(gcs_path):
                logging.info(f'GCS file already exists: {gcs_path}')
            else:
                logging.info(f'Starting to transfer file to GCS: {gcs_path}')
                # Transfer the local file to GCS.
                fs.put_file(
                    lpath=f"{self.local_storage}/{filename}",
                    rpath=gcs_path
                )    
                logging.info(f'Transferred file to GCS: {gcs_path}')

In [ ]:
# Transfer the files to Google Cloud Storage
t.to_gcs()

INFO:root:Starting to transfer file to GCS: hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_corr.tif
INFO:root:Transferred file to GCS: hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_corr.tif
INFO:root:Starting to transfer file to GCS: hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_los_disp.tif
INFO:root:Transferred file to GCS: hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_los_disp.tif
INFO:root:Starting to transfer file to GCS: hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_inc_map.tif
INFO:root:Transferred file to GCS: hyp3-da

## Create a GEE Asset

In [ ]:
#| export
@patch
def create_gee_asset(
    self:Transfer,
    # start_time: str,  # Starting time for the asset, as UTC string
    # end_time: str,  # Ending time for the asset, as UTC string
    # description,  # Description of the asset
):
    "Create an Earth Engine asset."

    ee.Initialize(project=self.gee_gcp_project)
    
    create_gee_image_collection(self.gee_gcp_project, self.gee_image_collection)
    # self._to_gcs()

    granule_names = self.job_dict['job_parameters']['granules']
    granules = asf_search.granule_search(granule_names)

    granule_times = [datetime.datetime.fromisoformat(x.properties['stopTime']) for x in granules]
    start_time = min(granule_times)
    end_time = max(granule_times)
    
    id = f"{self.job_dict['job_id']}"

    props = granules[0].properties
    description = (f"{props['platform']}"
                   f" - {props['processingLevel']}"
                   f" - {props['beamModeType']}")
    
    for file_dict in self.job_dict['files']:
        for band, filename in file_dict['extracted'].items():
            gcs_path = f'{self.gcs_bucket}/{filename}'
            print(gcs_path)
            
            request = {
                'type': 'IMAGE',
                'bands': {  # TODO: Update this once multi-band COG assets are supported
                    'id': band
                },
                'gcs_location': {
                    'uris': [f'gs://{gcs_path}']
                },
                'properties': {
                    'source':  file_dict['url'],
                    'band': band  # TODO: Remove this once multi-band COG assets are supported
                },
                'startTime': start_time.strftime(FORMAT_GEE_DATETIME_STRING),
                'endTime': end_time.strftime(FORMAT_GEE_DATETIME_STRING),
                'description': description
            }

            path_parts = [
                'projects',
                self.gee_gcp_project,
                'assets',
                self.gee_image_collection,
                # TODO: Remove the band suffix once multi-band COG assets are supported
                f'{id}_{band}'.replace(".", "_") 
            ]
            assetname = os.path.join(*[x for x in path_parts if x is not None])
            
            try:
                ee.data.createAsset(
                    value=request,
                    path=assetname
                )  
                logging.info('Finished creating a GEE asset.')
            except ee.EEException as e:
                if "does not exist or doesn't allow this operation" in str(e):
                    raise(e)
                else:
                    raise(e)  # TODO: Add logic to parse the EEException message.
                    logging.info('GEE asset already exists. Skipping.')

In [ ]:
t.create_gee_asset()

hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_corr.tif


INFO:root:Finished creating a GEE asset.


hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_los_disp.tif


INFO:root:Finished creating a GEE asset.


hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_inc_map.tif


...

INFO:root:Finished creating a GEE asset.


hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_wrapped_phase.tif


INFO:root:Finished creating a GEE asset.


hyp3-data-staging/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75/S1AA_20230331T140803_20230412T140803_VVP012_INT80_G_ueF_3F75_dem.tif


INFO:root:Finished creating a GEE asset.


Code Editor script for visualizing the asset:
https://code.earthengine.google.com/cdaf54611e608a4adb6bc9f6871666f4